In [ ]:
!git clone https://github.com/huggingface/diffusers

In [ ]:
%cd diffusers

In [ ]:
!pip install -e .

In [ ]:
!pip install -r examples/textual_inversion/requirements.txt

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key = "your_wandb_key") # replace with your wandb key

In [ ]:
!accelerate launch examples/textual_inversion/textual_inversion.py \
    --pretrained_model_name_or_path="stable-diffusion-v1-5/stable-diffusion-v1-5" \
    --train_data_dir="image_data_directory" \
    --learnable_property="object" \
    --placeholder_token="<irit_woman>" \
    --initializer_token="woman" \
    --resolution=512 \
    --train_batch_size=1 \
    --gradient_accumulation_steps=4 \
    --max_train_steps=1500 \
    --learning_rate=5.0e-05 \
    --scale_lr \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --output_dir="/kaggle/working/textual_inversion_irit" \
    --save_steps=500 \
    --report_to="wandb" \
    --logging_dir="logs" \
    --seed="0"

In [9]:
%cd /kaggle/working

/kaggle/working


In [10]:
!rm -rf diffusers

In [11]:
import sys

# Xoá module diffusers khỏi cache
if "diffusers" in sys.modules:
    del sys.modules["diffusers"]

# Loại bỏ tất cả đường dẫn trỏ đến local diffusers
sys.path = [p for p in sys.path if "diffusers" not in p]

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git --quiet

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import os
import numpy as np 
path_id = "stable-diffusion-v1-5/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(path_id).to("cuda")
repo_id_embeds = "/kaggle/working/textual_inversion_irit"
pipe.load_textual_inversion(repo_id_embeds)
prompt = "a professional photo portrait of <irit_woman> holding flowers, model photoshoot, professional photo, white background, Amazing Details, Best Quality"
negative_prompt = "asian,multiple heads, 2 heads, elongated body, double image, 2 faces, multiple people, double head, , (nsfw)+, naked, nude, porn, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up"
output_dir = "/kaggle/working/generated_images"
os.makedirs(output_dir, exist_ok=True)
for i in range(20):
    seed = np.random.randint(0, 999999)
    generator = torch.manual_seed(seed)
    image = pipe(prompt, negative_prompt = negative_prompt, num_inference_steps=100, guidance_scale=7.5, generator=generator).images[0]
    image.save(f"{output_dir}/image_{i+1:02d}_seed{seed}.png")